In [1]:
#path
import os

In [10]:
#os.environ['JAVA_HOME'] = '~/Data-Engineering_text-to-speech_data-collection/env/lib/python3.8/site-packages'

In [13]:
# setting up java the enviroment variable
os.environ['JAVA_HOME'] ='/usr/lib/jvm/java-11-openjdk-amd64'


In [14]:
import pyspark
import findspark
print(f'pyspark version: {pyspark.__version__}')
print(f'findspark version: {findspark.__version__}')


pyspark version: 3.3.0
findspark version: 2.0.1


In [15]:
# importing lib
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import Catalog
import pyspark.sql.functions as psf
from pyspark.sql.functions import lit, col,sum,avg,max,first,min,mean
import pyspark.ml as Pipeline
import pyspark.ml.pipeline as pmpip
import pyspark.ml.param as pmparam
from pyspark.ml.param.shared import HasInputCol,HasOutputCol
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,FloatType
from pyspark.sql.functions import monotonically_increasing_id 
from pyspark.sql.functions import struct,array,lit
from pyspark.streaming import StreamingContext

In [16]:
# creating configuration
spark_conf = SparkConf()
spark_conf.setAll([
    ('spark.master', 'local'),
    ('spark.app.name', 'text-speech-model'),
    ('spark.submit.deployMode', 'client'),
    ('spark.ui.showConsoleProgress', 'true'),
    ('spark.eventLog.enabled', 'false'),
    ('spark.logConf', 'false'),
    ('diriver_port', '8887'),
    ('cores_max','4'),
    ('ui_port','4040'),
    ('spark.driver.host', 'localhost'),
    ('spark.driver.memory','4g')
])

In [17]:
# Create connection 
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()

22/10/08 14:24:39 WARN Utils: Your hostname, ekubay-HP-Laptop-15-da0xxx resolves to a loopback address: 127.0.1.1; using 192.168.0.36 instead (on interface wlo1)
22/10/08 14:24:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/08 14:24:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [18]:
# the details 
spark

In [11]:

#current path
path = '../data/Amharic News Dataset.csv' #g1-raw-text-data-topic-dev 

In [19]:
# reading the data 
df_pyspark = spark.read.csv(path,header=True,inferSchema=True)

In [22]:
# here we can display texts
random_row_text = df_pyspark.rdd.takeSample(False, 2, seed=0)

In [23]:
# displaying the raw data
random_row_text

[Row(headline='የግድቡን መገንባት ስትደግፍ የነበረችው ሱዳን ኢትዮጵያ በመጭው ሀምሌ ወር ያቀደችውን የግድቡን የመጀመሪያ የውሃ ሙሌት አልደግፍም የሚል አቋም አንጸባርቃለች፡፡ ግብጽ በአንጻሩ ከግድቡ የመሰረት ድንጋይ መጣል ጀምሮ የሚደርሰኝ የውሃ መጠን ይቀንሳል በማለት ተቃውሞዋን እያሰማች ቆይታለች፤ የግድቡን የውሃ ሙሌቱ እቅድንም አትቀበለውም፡፡', category=None, date=None, views=None, article=None, link=None),
 Row(headline='ድርጅቱ ከባንኮች ጋር መሥራቱ አስተማማኝ የክፍያ ሥርዓት እንዲፈጠር አስችሏል', category='ቢዝነስ', date='June 21, 2017', views='Unknown', article='የኢትዮጵያ የምርት ገበያ ድርጅት ከባንኮች ጋር\xa0 በጋራ በመስራቱ በተገበያዮች ላይ የክፍያ መተማመን እንዲፈጠር ማስቻሉን አስታውቋል፡፡ምርት\xa0 ገበያው ከአሁን ቀደም ከአስር ባንኮች ጋር በጋር በመስራቱ ላለፉት ዘጠኝ ዓመታት\xa0 በሚያከናውነው የግብይት ሂደት ላይ \xa0መተማመንን \xa0የፈጠረ የክፍያ ሥርዓት እንዲኖር አስችሏል ብሏል፡፡በዛሬው ዕለት ምርት ገበያው ከብርሃን ኢንተርናሽናል ባንክ ጋር በጋር\xa0 ለመስራት የሚያስችለውን ስምምነት ተፈራርሟል፡፡የአሁኑን ስምምነት ተከትሎ ምርት ገበያው ደንበኞቹ በአስራ አንድ ባንኮች የክፍያ አገልግሎት መፈጸም\xa0 እንዲችሉ የሚያደርግ ነው፡፡በአሁኑ ወቅት ድርጅቱ በዘጠኝ የሀገሪቱ አካባቢዎች እየገነባቸው ባሉ የግብይት መፈፀሚያ ማዕከላት ሻጭና አቅራቢዎች የክፍያ አገልግሎት እንዲፈጽሙ የሚያስችል ነው ብለዋል ዋና ስራ አስፈፃሚው አቶ ኤርሚያስ እሸቱ፡፡\xa0', link='https://waltainfo.com/am/23211/')]

In [30]:
# import 
%matplotlib inline

In [35]:
# spark dataframe
df_pyspark.show(5)

+--------------------+--------+-----------------+-----+--------------------+--------------------+
|            headline|category|             date|views|             article|                link|
+--------------------+--------+-----------------+-----+--------------------+--------------------+
|የኦሊምፒክ ማጣሪያ ተሳታፊዎ...|    ስፖርት| January 14, 2021|    2|ብርሃን ፈይሳየኢትዮጵያ ቦክ...|https://www.press...|
|          አዲስ ዘመን ድሮ|    መዝናኛ|December 28, 2020|    4| የአዲስ ዘመን ጋዜጣ ቀደም...|                null|
|      መንገድ በመከተል አልፎ|    null|             null| null|                null|                null|
|      አልፎ በሚገኙት ሰፈሮች|    null|             null| null|                null|                null|
|       ብዙዎች የልኳንዳ ሥጋ|    null|             null| null|                null|                null|
+--------------------+--------+-----------------+-----+--------------------+--------------------+
only showing top 5 rows



### Transforming the dataset 

In [40]:
# filltering and saving non null values records only to S3 bucket based on the 3 columns
df_pyspark3=df_pyspark.filter(df_pyspark.headline.isNotNull() & df_pyspark.category.isNotNull() & df_pyspark.article.isNotNull())

In [41]:
# checking  
df_pyspark3.show(2)

+--------------------+--------+-----------------+-----+--------------------+--------------------+
|            headline|category|             date|views|             article|                link|
+--------------------+--------+-----------------+-----+--------------------+--------------------+
|የኦሊምፒክ ማጣሪያ ተሳታፊዎ...|    ስፖርት| January 14, 2021|    2|ብርሃን ፈይሳየኢትዮጵያ ቦክ...|https://www.press...|
|          አዲስ ዘመን ድሮ|    መዝናኛ|December 28, 2020|    4| የአዲስ ዘመን ጋዜጣ ቀደም...|                null|
+--------------------+--------+-----------------+-----+--------------------+--------------------+
only showing top 2 rows



In [42]:
# saving to [S3] bucket

df_pyspark3.write.option("header",True).csv("../data/cleanNews")
print('file successfully transformed')


file successfully transformed


**Streaming Text**

Now after generating the random text, then let the streaming begin and we can begin to immediately stream the data into the frontend for the user to see. For this service we shall use apache kafka. And we shall send the publish the random text data, keeping in mind that we had uniquely generated the uuid and we configure this with the api, therefore there shall be no overlaps with the data, so once we have received it. We consume the text and send back an audio file that we are going to carefully but keenly scrutinize before funishing it to the data lake that is our s3 bucket


In [24]:
# streaming text
!pip install kafka-python

  Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)


In [25]:
# streaming text
from kafka import KafkaConsumer,KafkaProducer,KafkaAdminClient
from kafka.admin import NewTopic

In [26]:
# creating id 
import uuid

text = {
    "id":str(uuid.uuid4())
}
#A universally unique identifier (UUid) is a 128-bit label used for information in computer systems

In [43]:
#creating clients
# client = KafkaAdminClient(bootstrap_servers=['127.0.0.1:9092'],
#                          client_id='admin-client')

In [ ]:
# text update
text.update({'headline':random_row_text[0]['headline']})
text.update({'article':random_row_text[0]['article']})
text.update({'audio':'../data/test_amharic.wav'})

In [ ]:
# creating topic
topic_list = []
topic_list.append(
    NewTopic(name='spark-transformed-text',
              num_partitions=1,
              replication_factor=1)
)